# 3. KOBIS OPEN API

영화관 입장권 통합 전산망 OPENAPI를 사용하여 일일 박스오피스 자료를 가져오고 JSON파일로 저장

##  1) 일별 박스오피스
kobis에서 일별박스오피스 api를 활용하여 movieCd, movieNm, openDt, salesAmt, salesShare, salesInten, salesChange, salesAcc, audiCnt, audiInten, audiChange, audiAcc, scrnCnt, showCnt, CurrentDate, Nation 을 추출

##  2) 영화 상세정보
kobis에서 영화상세정보 api를 활용하여 movieCd, movieNm, movieNmEn, movieNmOg, showTm, prdtYear, openDt, prdtStatNm, typeNm, nations, genres, directors, actors 를 추출 = movie_info.txt


.

.

# 3.1 일별 박스오피스

### 1) kobis API를 통해 일별 박스오피스 데이터를 받아온다. 
###     파일은' 해당일자'를 이름으로 지정하고
###     JSON 타입으로 저장한다.


In [1]:
import os
import requests
import json
import calendar
from tqdm import tqdm
from tqdm import tqdm_notebook
from datetime import date, timedelta

key='ad816991534afeaef71f07e7336b0d61'
movie_info_path='/home/ubuntu/movie/'

In [2]:
def daily_box_office_to_json(start_date,end_date):    
    tmp_date = start_date
#     startDt = int(str(start_date)[0:4]+str(start_date)[5:7]+str(start_date)[8:10])
#     endDt = int(str(end_date)[0:4]+str(end_date)[5:7]+str(end_date)[8:10])
    days = end_date - start_date # 날짜 경과
    for i in tqdm_notebook(range(days.days+1)):
        targetDt = int(str(tmp_date)[0:4]+str(tmp_date)[5:7]+str(tmp_date)[8:10])
        
        if(os.path.isfile(movie_info_path+'raw_data/'+str(targetDt)+'.json')!=1):
            url='http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key='+key+'&targetDt='+str(targetDt)
            res = requests.get(url)
            text = res.text
            d = json.loads(text)
            with open(movie_info_path+'raw_data/daily'+'/'+str(targetDt)+'.json', 'w', encoding="utf-8") as make_file:
                json.dump(d, make_file, ensure_ascii=False, indent="\t")
        tmp_date += timedelta(1)

### 3) daliy_box_office_to_json 사용자 함수 실행한다.

In [3]:
# 2017년 01월 ~ 2018년 02월까지 boxoffice 를 받아온다.
start_date = date(2017,1,1)
end_date = date(2018,2,28)
daily_box_office_to_json(start_date,end_date)

### 4) 일별 데이터를 분석할 때 용이하도록 월별로 합친다.

In [4]:
import pandas as pd
import requests
from datetime import datetime
from datetime import timedelta
import json
from tqdm import tqdm_notebook
from dateutil.relativedelta import relativedelta

key='ad816991534afeaef71f07e7336b0d61'
movie_info_path='/home/ubuntu/movie/'

In [5]:
def daily_boxoffice(start_date, end_date) :
    
    final_list = []

    for single_date in tqdm_notebook(pd.date_range(start_date, end_date)) :
        targetDt = str(single_date)[0:4]+str(single_date)[5:7]+str(single_date)[8:10]
        with open(movie_info_path+'raw_data/daily/'+str(targetDt)+'.json', 'r', encoding="utf-8") as read_file:
            d = json.load(read_file)

        for item in d["boxOfficeResult"]["dailyBoxOfficeList"] :
            value_list = []
            key_list = []
            for key, value in item.items() :
                key_list.append(key)
                value_list.append(value)

            value_list.append(single_date)
            key_list.append('CurrentDate')
            final_list.append(value_list)

    return pd.DataFrame(final_list, columns=key_list)

In [6]:
my_date = date(2017, 1, 1)
movie_code_list_all=pd.DataFrame()
for i in tqdm_notebook(range(14)): # 14개월 추출이라서 range(14)
    start_date = my_date + relativedelta(months=i)
    end_date = start_date + relativedelta(months=1) - timedelta(days=1)
    
    daily_boxoffice_data = daily_boxoffice(start_date, end_date)
    startdate=start_date.strftime('%Y%m%d')
    enddate=end_date.strftime('%Y%m%d')
    
    movie_code_list = pd.DataFrame({'movieCd' :daily_boxoffice_data.movieCd.drop_duplicates(), 'movieNm':daily_boxoffice_data.movieNm.drop_duplicates(), 'director':'','Rate' : 0})
    
    
    daily_boxoffice_data.to_csv(movie_info_path+"data/"+"daily_"+startdate+"~"+enddate+".csv", index=False)
    
    
    movie_code_list_all = movie_code_list.append(movie_code_list_all)
    movie_code_list_all = movie_code_list_all.drop_duplicates()
    movie_code_list_all.to_csv(movie_info_path+"data/"+"movie_code_list_all.csv", index=False)

In [7]:
movie_code_list_all.head()

,Rate,director,movieCd,movieNm
0,0,,20168323,염력
1,0,,20176104,그것만이 내 세상
2,0,,20178126,인시디어스4: 라스트 키
3,0,,20179985,코코
4,0,,20150976,신과함께-죄와 벌


### 5) 일일 데이터를 하나로 합치기


In [ ]:
# start_date = date(2017,1,1)
# end_date = date(2018,2,28)
# a = daily_boxoffice(start_date, end_date)
# a

# 3.2 영화 상세정보 

### 1) kobis API를 통해 영화 정보 데이터를 받아온다.
### 파일 이름은 movie_info 이고
### csv 파일 형식으로 저장한다.

In [8]:
#영화데이터  : txt 로 저장 함수 정의
def movie_data(lists) :    
    dict_list = dict()
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'
    
    for movie in lists :
        #print(movie)
        dic={ 'key' : key, 'movieCd' : movie}        
        req = requests.get(url, params = dic)
        
        dict_list[movie] = req.json()['movieInfoResult']['movieInfo'] #dict_list[movie] 가 있어야 append형식으로 됨. 없으면 마지막것만 추출
    return dict_list

In [9]:
movie_info=movie_data(movie_code_list_all.movieCd) 
with open(movie_info_path+'raw_data/movie_info/movie_info.txt', 'w', encoding='utf-8') as outfile:   #딕셔너리를 json으로 저장
    json.dump(movie_info, outfile, ensure_ascii=False)

### 2)  movie_code_list_all 에 dirctors 삽입

In [10]:
movie_code_list_all.reset_index(drop=True, inplace=True)
movie_code_list_all

,Rate,director,movieCd,movieNm
0,0,,20168323,염력
1,0,,20176104,그것만이 내 세상
2,0,,20178126,인시디어스4: 라스트 키
3,0,,20179985,코코
4,0,,20150976,신과함께-죄와 벌
5,0,,20175054,메이즈 러너: 데스 큐어
6,0,,20178395,12 솔져스
7,0,,20170590,1987
8,0,,20179228,마야2
9,0,,20179205,조선명탐정: 흡혈괴마의 비밀


In [11]:
#movie_info의  director를 뽑아서 movie_code_list에 넣기
movie_info_keys=list(movie_info.keys())
#movie_info_keys

In [12]:
for i in range(len(movie_code_list_all)):
    with open(movie_info_path+'raw_data/movie_info/movie_info.txt', 'r', encoding="utf-8") as make_file:
        d = json.load(make_file)
    if(d[movie_info_keys[i]]['directors'] != []):
        movie_code_list_all['director'][i]=d[movie_info_keys[i]]['directors'][0]['peopleNm']
        
    else:
        movie_code_list_all['director'][i] = '---'

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
movie_code_list_all

,Rate,director,movieCd,movieNm
0,0,연상호,20168323,염력
1,0,최성현,20176104,그것만이 내 세상
2,0,애덤 로비텔,20178126,인시디어스4: 라스트 키
3,0,리 언크리치,20179985,코코
4,0,김용화,20150976,신과함께-죄와 벌
5,0,웨스 볼,20175054,메이즈 러너: 데스 큐어
6,0,니콜라이 퓰시,20178395,12 솔져스
7,0,장준환,20170590,1987
8,0,노엘 클리어리,20179228,마야2
9,0,김석윤,20179205,조선명탐정: 흡혈괴마의 비밀


In [14]:
with open('movie_info.txt', 'w', encoding='utf-8') as outfile:   #딕셔너리를 json으로 저장
    json.dump(movie_info, outfile, ensure_ascii=False)